# pytorch 加载 CNN-F/VGG-F 参数
- [tensorflow加载CNN-F/VGG-F预训练参数](https://blog.csdn.net/HackerTom/article/details/103189798)
- [test.cnnf.ipynb](https://download.csdn.net/download/HackerTom/12170872)
- [DCMH-CVPR2017/DCMH_tensorflow/DCMH_tensorflow/net_structure_img.py](https://github.com/jiangqy/DCMH-CVPR2017/blob/master/DCMH_tensorflow/DCMH_tensorflow/net_structure_img.py)
- [PyTorch中文文档](https://pytorch-cn.readthedocs.io/zh/latest/)

In [1]:
import scipy.io as sio
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
CNN_F = "vgg_net.mat"
cnnf = sio.loadmat(CNN_F)

In [3]:
net = cnnf["net"]
net = net[0][0]

In [4]:
layers = net[0][0]

# process functions

In [5]:
def make_conv(layer):
    """pytorch: (n, C, h, w)
    tf: (n, h, w, C)
    src: https://github.com/pytorch/pytorch/blob/master/torch/nn/modules/conv.py#L329
    """
    print("name:", layer[0])
    print("type:", layer[1])
    k, b = layer[2][0]
    #b = b.flatten()
    print("kernel:", k.shape, ", bias:", b.shape)
    shape = layer[3][0]
    print("shape:", shape)
    pad = layer[4][0]
    print("pad:", pad)
    stride = layer[5][0]
    print("stride:", stride)
    
    conv = nn.Conv2d(shape[2], shape[3], shape[:2],
                     stride=tuple(stride))  # must convert to tuple
                    #  padding=tuple(pad))
    print(conv.weight.data.size())
    print(conv.bias.data.size())
    conv.weight.data = torch.from_numpy(k.transpose((3, 2, 0, 1)))
    #print(conv.bias.data)
    conv.bias.data = torch.from_numpy(b.flatten())
    #print(conv.bias.data)
    
    if np.sum(pad) > 0:
        padding = nn.ZeroPad2d(tuple(pad.astype(np.int32)))
        conv = nn.Sequential(padding, conv)

    return conv  

In [6]:
class LRN(nn.Module):
    """ref:
    - https://zhuanlan.zhihu.com/p/29786939
    - https://www.jianshu.com/p/c06aea337d5d
    """
    def __init__(self, local_size=1, bias=1.0, alpha=1.0, beta=0.75, ACROSS_CHANNELS=False):
        super(LRN, self).__init__()
        self.ACROSS_CHANNELS = ACROSS_CHANNELS
        if self.ACROSS_CHANNELS:
            self.average=nn.AvgPool3d(kernel_size=(local_size, 1, 1), #0.2.0_4会报错，需要在最新的分支上AvgPool3d才有padding参数
                    stride=1,
                    padding=(int((local_size-1.0)/2), 0, 0)) 
        else:
            self.average=nn.AvgPool2d(kernel_size=local_size,
                    stride=1,
                    padding=int((local_size-1.0)/2))

        self.bias = bias
        self.alpha = alpha
        self.beta = beta
    
    def forward(self, x):
        if self.ACROSS_CHANNELS:
            div = x.pow(2).unsqueeze(1)
            div = self.average(div).squeeze(1)
            div = div.mul(self.alpha).add(self.bias).pow(self.beta)#这里的1.0即为bias
        else:
            div = x.pow(2)
            div = self.average(div)
            div = div.mul(self.alpha).add(self.bias).pow(self.beta)
        x = x.div(div)
        return x

In [7]:
def make_lrn(layer):
    print("name:", layer[0])
    print("type:", layer[1])
    param = layer[2][0]
    print("local_size/depth_radius:", param[0])
    print("bias:", param[1])
    print("alpha:", param[2])
    print("beta:", param[3])
    
    lrn = LRN(int(param[0]), param[1], param[2], param[3])
    return lrn

In [8]:
def make_pool(layer):
    print("name:", layer[0])
    print("type:", layer[1])
    print("pool type:", layer[2])
    k_size = layer[3][0]
    print("kernel size:", k_size)
    stride = layer[4][0]
    print("stride:", stride)
    pad = layer[5][0]
    print("pad:", pad)
    
    padding = nn.ZeroPad2d(tuple(pad.astype(np.int32)))
    pool = nn.MaxPool2d(k_size,
                        stride=tuple(stride))
                        # padding=tuple(pad))
    return nn.Sequential(padding, pool)

# 1st block

In [10]:
i = 1
print(i, "layer")
conv1 = make_conv(layers[i-1][0][0])
print(conv1)

1 layer
name: ['conv1']
type: ['conv']
kernel: (11, 11, 3, 64) , bias: (64, 1)
shape: [11 11  3 64]
pad: [0 0 0 0]
stride: [4 4]
torch.Size([64, 3, 11, 11])
torch.Size([64])
Conv2d(3, 64, kernel_size=[11 11], stride=(4, 4))


In [11]:
i = 2
print(i, "layer")
lay = layers[i-1]
# show(lay)
lay = lay[0][0]
print(lay.shape, type(lay), len(lay))
print("name:", lay[0])
print("type:", lay[1])

relu1 = nn.ReLU(inplace=True)
print(relu1)

2 layer
() <class 'numpy.void'> 4
name: ['relu1']
type: ['relu']
ReLU(inplace=True)


In [12]:
i = 3
print(i, "layer")
lrn1 = make_lrn(layers[i-1][0][0])
print(lrn1)

3 layer
name: ['norm1']
type: ['lrn']
local_size/depth_radius: 5.0
bias: 2.0
alpha: 0.0001
beta: 0.75
LRN(
  (average): AvgPool2d(kernel_size=5.0, stride=1, padding=2)
)


In [13]:
i = 4
print(i, "layer")
pool1 = make_pool(layers[i-1][0][0])
print(pool1)

4 layer
name: ['pool1']
type: ['pool']
pool type: ['max']
kernel size: [3 3]
stride: [2 2]
pad: [0 1 0 1]
MaxPool2d(kernel_size=[3 3], stride=(2, 2), padding=(0, 1, 0, 1), dilation=1, ceil_mode=False)


# 2nd block

In [14]:
i = 5
print(i, "layer")
conv2 = make_conv(layers[i-1][0][0])
print(conv2)

5 layer
name: ['conv2']
type: ['conv']
kernel: (5, 5, 64, 256) , bias: (256, 1)
shape: [  5   5  64 256]
pad: [2 2 2 2]
stride: [1 1]
torch.Size([256, 64, 5, 5])
torch.Size([256])
Conv2d(64, 256, kernel_size=[5 5], stride=(1, 1), padding=(2, 2, 2, 2))


In [15]:
i = 6
print(i, "layer")
lay = layers[i-1]
# show(lay)
lay = lay[0][0]
print(lay.shape, type(lay), len(lay))
print("name:", lay[0])
print("type:", lay[1])

relu2 = nn.ReLU(inplace=True)
print(relu2)

6 layer
() <class 'numpy.void'> 4
name: ['relu2']
type: ['relu']
ReLU(inplace=True)


In [16]:
i = 7
print(i, "layer")
lrn2 = make_lrn(layers[i-1][0][0])
print(lrn2)

7 layer
name: ['norm2']
type: ['lrn']
local_size/depth_radius: 5.0
bias: 2.0
alpha: 0.0001
beta: 0.75
LRN(
  (average): AvgPool2d(kernel_size=5.0, stride=1, padding=2)
)


In [17]:
i = 8
print(i, "layer")
pool2 = make_pool(layers[i-1][0][0])
print(pool2)

8 layer
name: ['pool2']
type: ['pool']
pool type: ['max']
kernel size: [3 3]
stride: [2 2]
pad: [0 1 0 1]
MaxPool2d(kernel_size=[3 3], stride=(2, 2), padding=(0, 1, 0, 1), dilation=1, ceil_mode=False)


# 3rd block

In [18]:
i = 9
print(i, "layer")
conv3 = make_conv(layers[i-1][0][0])
print(conv3)

9 layer
name: ['conv3']
type: ['conv']
kernel: (3, 3, 256, 256) , bias: (256, 1)
shape: [  3   3 256 256]
pad: [1 1 1 1]
stride: [1 1]
torch.Size([256, 256, 3, 3])
torch.Size([256])
Conv2d(256, 256, kernel_size=[3 3], stride=(1, 1), padding=(1, 1, 1, 1))


In [19]:
i = 10
print(i, "layer")
lay = layers[i-1]
# show(lay)
lay = lay[0][0]
print(lay.shape, type(lay), len(lay))
print("name:", lay[0])
print("type:", lay[1])

relu3 = nn.ReLU(inplace=True)
print(relu3)

10 layer
() <class 'numpy.void'> 4
name: ['relu3']
type: ['relu']
ReLU(inplace=True)


# 4th block

In [20]:
i = 11
print(i, "layer")
conv4 = make_conv(layers[i-1][0][0])
print(conv4)

11 layer
name: ['conv4']
type: ['conv']
kernel: (3, 3, 256, 256) , bias: (256, 1)
shape: [  3   3 256 256]
pad: [1 1 1 1]
stride: [1 1]
torch.Size([256, 256, 3, 3])
torch.Size([256])
Conv2d(256, 256, kernel_size=[3 3], stride=(1, 1), padding=(1, 1, 1, 1))


In [21]:
i = 12
print(i, "layer")
lay = layers[i-1]
# show(lay)
lay = lay[0][0]
print(lay.shape, type(lay), len(lay))
print("name:", lay[0])
print("type:", lay[1])

relu4 = nn.ReLU(inplace=True)
print(relu4)

12 layer
() <class 'numpy.void'> 4
name: ['relu4']
type: ['relu']
ReLU(inplace=True)


# 5th block

In [22]:
i = 13
print(i, "layer")
conv5 = make_conv(layers[i-1][0][0])
print(conv5)

13 layer
name: ['conv5']
type: ['conv']
kernel: (3, 3, 256, 256) , bias: (256, 1)
shape: [  3   3 256 256]
pad: [1 1 1 1]
stride: [1 1]
torch.Size([256, 256, 3, 3])
torch.Size([256])
Conv2d(256, 256, kernel_size=[3 3], stride=(1, 1), padding=(1, 1, 1, 1))


In [23]:
i = 14
print(i, "layer")
lay = layers[i-1]
# show(lay)
lay = lay[0][0]
print(lay.shape, type(lay), len(lay))
print("name:", lay[0])
print("type:", lay[1])

relu5 = nn.ReLU(inplace=True)
print(relu5)

14 layer
() <class 'numpy.void'> 4
name: ['relu5']
type: ['relu']
ReLU(inplace=True)


In [24]:
i = 15
print(i, "layer")
pool5 = make_pool(layers[i-1][0][0])
print(pool5)

15 layer
name: ['pool5']
type: ['pool']
pool type: ['max']
kernel size: [3 3]
stride: [2 2]
pad: [0 1 0 1]
MaxPool2d(kernel_size=[3 3], stride=(2, 2), padding=(0, 1, 0, 1), dilation=1, ceil_mode=False)


# 6th block

In [25]:
i = 16
print(i, "layer")
fc6 = make_conv(layers[i-1][0][0])
print(fc6)

16 layer
name: ['fc6']
type: ['conv']
kernel: (6, 6, 256, 4096) , bias: (4096, 1)
shape: [   6    6  256 4096]
pad: [0 0 0 0]
stride: [1 1]
torch.Size([4096, 256, 6, 6])
torch.Size([4096])
Conv2d(256, 4096, kernel_size=[6 6], stride=(1, 1))


In [26]:
i = 17
print(i, "layer")
lay = layers[i-1]
# show(lay)
lay = lay[0][0]
print(lay.shape, type(lay), len(lay))
print("name:", lay[0])
print("type:", lay[1])

relu6 = nn.ReLU(inplace=True)
print(relu6)

17 layer
() <class 'numpy.void'> 4
name: ['relu6']
type: ['relu']
ReLU(inplace=True)


# 7th block

In [27]:
i = 18
print(i, "layer")
fc7 = make_conv(layers[i-1][0][0])
print(fc7)

18 layer
name: ['fc7']
type: ['conv']
kernel: (1, 1, 4096, 4096) , bias: (4096, 1)
shape: [   1    1 4096 4096]
pad: [0 0 0 0]
stride: [1 1]
torch.Size([4096, 4096, 1, 1])
torch.Size([4096])
Conv2d(4096, 4096, kernel_size=[1 1], stride=(1, 1))


In [28]:
i = 19
print(i, "layer")
lay = layers[i-1]
# show(lay)
lay = lay[0][0]
print(lay.shape, type(lay), len(lay))
print("name:", lay[0])
print("type:", lay[1])

relu7 = nn.ReLU(inplace=True)
print(relu7)

19 layer
() <class 'numpy.void'> 4
name: ['relu7']
type: ['relu']
ReLU(inplace=True)
